In [14]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [15]:
# Disable warnings
tf.get_logger().setLevel('ERROR')

# Turn off GPU usage for tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '2'

In [16]:
def prep_data(raw_df):
  # preparing the data and obtaining necessary information
  raw_data = raw_df.values
  X = raw_data[:, :-1]
  y = raw_data[:, -1]

  print(X.shape, y.shape)

  # performing the train_test_split
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify=y)

  # standardizing the features
  scaler = StandardScaler()
  X_train_normalized = scaler.fit_transform(X_train)
  X_test_normalized = scaler.transform(X_test)

  return X_train_normalized, X_test_normalized, y_train, y_test

In [17]:
def build_model(num_features):
  # creating the Neural Network
  classifier = keras.Sequential()
  classifier.add(layers.Dense(64, input_dim=(num_features), activation="relu"))
  classifier.add(layers.Dense(32, activation="relu"))
  classifier.add(layers.Dense(8, activation="softmax"))

  classifier.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return classifier

In [18]:
def main():
  raw_df = pd.read_csv("./cover_data.csv")
  columns = raw_df.columns.tolist()

  features = columns[:-1]
  label = columns[-1]

  X_train, X_test, y_train, y_test = prep_data(raw_df)

  num_features = len(features)
  model = build_model(num_features)

  print("Summary Report of Model")
  model.summary()

  num_epochs = 100
  batch_size = 1024

  earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=3)
  history = model.fit(X_train,
                        y_train,
                        epochs=num_epochs,
                        batch_size=batch_size,
                        callbacks=[earlystop_callback],
                        validation_split=0.1,
                        verbose=1)

  score = model.evaluate(X_test, y_test, verbose=0)
  print(f'Test loss: {score[0]}')
  print(f'Test accuracy: {score[1]}')

  y_pred = np.argmax(model.predict(X_test), axis=-1)

  class_names = ['Spruce/Fir', 'Lodgepole Pine',
                   'Ponderosa Pine', 'Cottonwood/Willow',
                   'Aspen', 'Douglas-fir', 'Krummholz']
  print(classification_report(y_test, y_pred, target_names=class_names))


In [20]:
if __name__ == '__main__':
    main()

(581012, 54) (581012,)
Summary Report of Model
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                3520      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 8)                 264       
                                                                 
Total params: 5864 (22.91 KB)
Trainable params: 5864 (22.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
409/409 [==============================] - 5s 6ms/step - loss: 0.7870 - accuracy: 0.6803 - val_loss: 0.6127 - val_accuracy: 0.7397
Epoch 2/100
409/409 [==============================] - 2s 5ms/step - loss: 0.5827 - accuracy: 